In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
train_data = pd.read_csv('/content/gdrive/MyDrive/CS274/CS274H04/hw4_naive.csv')
y_val = train_data['Label']
train_data.drop(labels = ['Label'], axis=1,inplace=True)
X_val = train_data
X_train, X_test, y_train, y_test = train_test_split(X_val, y_val, test_size=0.2,random_state = 5)

In [ ]:
#def laplace_smoothing(matched_counts, ttl_class, feat_n, cls_n, alpha=1):
    #print(matched_counts)
#    probs = np.zeros_like(matched_counts)


#    for i in range(cls_n):
#      probs[i, :] = (matched_counts[i, :] + alpha) / (ttl_class[i] + cls_n )
#    return probs


In [ ]:
#def fit(xtrain, ytrain, classes, alpha = 1):
  #print(xtrain.shape)
#  print(ytrain)
#  feat_n = xtrain.shape[1] #feature num
#  cls_n = len(classes) #number of classes
# cls_c = ytrain.value_counts().to_dict() #element num in each class

#  mtchd_c = np.zeros((cls_n, feat_n))
#  print(mtchd_c.shape)

#  for i, cls in enumerate(classes):
#    mtchd_d = xtrain[ytrain == cls]
#    print(mtchd_d.shape)
#    for idx in range(feat_n):
#        #print(mtchd_d.iloc[:, idx])
#      mtchd_c[i][idx] = mtchd_d.iloc[:, idx].sum()/xtrain.shape[0]
#  #print(mtchd_c.shape)

#  return laplace_smoothing(mtchd_c, cls_c, feat_n, cls_n, 1)






In [ ]:
#def prdct(xtest, classes, probs, alpha = 1 ):
#  num_features = xtest.shape[1]
#  prdcts = np.zeros(xtest.shape[0])
#  for idx in range(xtest.shape[0]):
#    scores = np.zeros(len(classes))

#    for i, label in enumerate(classes):
#      label_index = np.where(classes == label)
#      score = 0
#      for feature_idx in range(num_features):
#          feature_value = xtest.iloc[idx,feature_idx]
#          if probs[label_index, feature_idx] < 0:
#            probs[label_index, feature_idx] = 8 +probs[label_index, feature_idx]
#          score += np.log(probs[label_index, feature_idx]) * feature_value
#      #print(score)
#      scores[i] = score
#    #print(scores)
#    prdct = classes[np.argmax(scores)]
#    prdcts[idx] =  prdct
#  return prdcts


In [4]:
def Mnbfit(xtrain,ytrain):
  classes = [0,1]
  count_classes = []
  ytrain = ytrain.to_numpy()
  for i, cls in enumerate(classes):
    #print(cls)
    count_class = xtrain[ytrain == cls]
    count_classes.append(count_class.values)
  count_classes = np.array(count_classes)
  priors = []
  prior0 = np.sum(len(count_classes[0])/len(ytrain))
  priors.append(prior0)
  prior1 = np.sum(len(count_classes[1])/len(ytrain))
  priors.append(prior0)
  return (count_classes), priors
fitted, priorz = Mnbfit(X_train, y_train)
#print(fitted[0])

<ipython-input-4-7569c6b46cbf>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  count_classes = np.array(count_classes)


In [5]:
def Mnbprd(xtest, count_class, priors, alpha):
  preds = []
  for i in range(len(xtest)):
    pred0 = np.prod([(np.sum((count_class[0]==xtest.iloc[i].values))+alpha)/(len(count_class[0])+alpha*xtest.shape[1])])
    pred1 = np.prod([(np.sum((count_class[1]==xtest.iloc[i].values))+alpha)/(len(count_class[1])+alpha*xtest.shape[1])])
    preds.append(0 if (pred0*priors[0]) > (pred1*priors[1])else 1)
  return preds

y_pred_mnb = Mnbprd(X_test, fitted, priorz,1)

In [128]:
print(f"accuracy: {accuracy_score(y_test, y_pred_mnb)}")
print(f"f1Score: {f1_score(y_test, y_pred_mnb, average = 'macro')}")

accuracy: 0.7758928571428572
f1Score: 0.7561075063397994


In [7]:
def Gnbfit(xtrain, ytrain):
  classes = [0,1]
  count_classes = []
  class_mius = []
  class_sigmas = []
  class_priors = []
  for c in classes:
    X_c = xtrain[ytrain == c]
    class_mius.append(X_c.mean(axis = 0).values)
    class_sigmas.append(X_c.std(axis = 0).values )
    class_priors.append(np.sum(((X_c) / len(xtrain)).values))
  return class_mius, class_sigmas, class_priors
mius, sigmas, priors = Gnbfit(X_train, y_train)

In [8]:
def Gnbprd(xtest, miu, sigma, prior):
  preds = []
  classes = [0,1]
  #print(miu)
  for x in xtest.values:
    pred0 = (np.prod((1 / (np.sqrt(2 * np.pi) * sigma[0])) * np.exp(-((x - miu[0]) ** 2) / (2 * (sigma[0] ** 2))))).sum()
    pred1 = (np.prod((1 / (np.sqrt(2 * np.pi) * sigma[1])) * np.exp(-((x - miu[1]) ** 2) / (2 * (sigma[1] ** 2))))).sum()
    #print(prior[0].shape)
    #print(xtest.values.shape)
    preds.append(0 if (pred0*prior[0]) > (pred1*prior[1]) else 1)
  return preds

y_pred_gnb = Gnbprd(X_test, mius, sigmas, priors)

In [129]:
print(f"accuracy: {accuracy_score(y_test, y_pred_gnb)}")
print(f"f1Score: {f1_score(y_test, y_pred_gnb, average = 'macro')}")

accuracy: 0.5991071428571428
f1Score: 0.5309265897501192


In [10]:
ori_data = pd.read_csv('/content/gdrive/MyDrive/CS274/CS274H04/hw4_cluster.csv')
colnames = list(ori_data.columns[1:-1])
ori_data = ori_data.values

In [122]:
def initiate_centroids(K, dset, initialization):
  if initialization == 'random':
    np.random.shuffle(dset)
    return dset[:K]
  elif initialization == 'random_seed':
    np.random.seed(4)  # Set a seed for reproducibility
    random_indices = np.random.choice(len(dset), K, replace=False)
    #print(dset.values[random_indices])
    return dset[random_indices]

In [12]:
def calculate_distance(point, centroid):
  #print(point)
  #print(centroid)
  return np.square(np.sum((point- centroid)**2))

In [13]:
def centroid_assignation(clusters, centroid_method):
  centroids = []
  if centroid_method == 'mean':
    for cluster in clusters:
      centroids.append(np.mean(cluster, axis=0))
  return np.array(centroids)

In [59]:
def k_means(data, K,  initialization, max_iter, centorid_method = 'mean'):
  centroids = initiate_centroids( K,data, initialization)
  #print(centroids)
  for c in range(max_iter):
    clusters = [[] for c in range(K)]
    last_clusters = None
    for point in data:
      distances = [calculate_distance(point, centroid) for centroid in centroids]
      closest_centroid = np.argmin(distances)
      clusters[closest_centroid].append(np.array(point))
    if last_clusters is not None and np.array_equal(clusters, last_clusters):
      break
    last_clusters = clusters.copy()
    centroids = centroid_assignation(clusters, centorid_method)
  return clusters


In [124]:
from sklearn.metrics import silhouette_samples #for test purpose, for implement from scrach one see below

def calculate_silhouette_score(clusters):
    data = np.concatenate(clusters)
    labels = np.concatenate([[i]*len(cluster) for i, cluster in enumerate(clusters)])
    silhouette_vals = silhouette_samples(data, labels)
    mean_silhouette = np.mean(silhouette_vals)
    return mean_silhouette


In [118]:
clusters = k_means(ori_data, 5, 'random_seed', 50 )

silhouettescore = calculate_silhouette_score(clusters)
print(f"silhouettescore:{silhouettescore}")

silhouettescore:0.5900989263516132


In [113]:
def silhouette(clusters):

  scores = []
  for i, pointS in enumerate(clusters):
    for point in pointS:
      #current_label = labels[i]
      if len(pointS) > 1:
        a = np.sum([(point-q)**2 for q in pointS])/ (len(pointS)-1)
        #print(a)
      else:
        a = 0

      b = np.inf
      for j, other_cluster in enumerate(clusters):
        if j != i:
          mean_dist = np.mean([(point-q)**2 for q in other_cluster])
          if mean_dist < b:
            b = mean_dist


    silhouette = (b - a) / max(a, b) if max(a, b) > 0 else 0
    scores.append(silhouette)
  return np.mean(scores)

In [123]:
clusters = k_means(ori_data, 5, 'random_seed', 50 )
silhouettescore = silhouette(clusters)
print(f"silhouettescore:{silhouettescore}")

silhouettescore:0.5167196842408075
